In [2]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vectorizer, HeadQA, HeadQA_IR, clean_words, parse_dataset, parse_ir_dataset, random_oversamplig, save_dataset_to_pickle, load_dataset_from_pickle
from training import train, validate, evaluate, train_ir, validate_ir, load_embeddings_from_file, make_embedding_matrix


import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


In [4]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [5]:
#training_instances = parse_ir_dataset(training)
#validation_instances = parse_ir_dataset(validation)
#testing_instances = parse_ir_dataset(testing)

#oversampled_training = random_oversamplig(training_instances)

#save_dataset_to_pickle('../data/training_ir.pickle', training_instances)
#save_dataset_to_pickle('../data/validation_ir.pickle', validation_instances)
#save_dataset_to_pickle('../data/testing_ir.pickle', testing_instances)
#save_dataset_to_pickle('../data/oversampled_training_ir.pickle', oversampled_training)

In [6]:
training_instances = load_dataset_from_pickle('../data/training_ir.pickle')
validation_instances = load_dataset_from_pickle('../data/validation_ir.pickle')
testing_instances = load_dataset_from_pickle('../data/testing_ir.pickle')
oversampled_training = load_dataset_from_pickle('../data/oversampled_training_ir.pickle')

In [7]:
oversampled_training[0]

{'question': 'Los potenciales postsinápticos excitadores:',
 'answer': 'Son de tipo todo o nada.',
 'tok_qtext': ['Los', 'potenciales', 'postsinápticos', 'excitadores', ':'],
 'tok_atext': ['Son', 'de', 'tipo', 'todo', 'o', 'nada', '.'],
 'label': 0,
 'category': 'biology'}

In [8]:
vectorizer = Vectorizer.vectorize_ir_dataset(oversampled_training)

In [9]:
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [10]:
trainset = HeadQA_IR(instances=training_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
validset = HeadQA_IR(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
testset = HeadQA_IR(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=15)

In [11]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [59]:
class LSTM_QA(torch.nn.Module):
    def __init__(self, vocab_size, hidden_size, x_size, n_classes, embedding_size=300,
                 padding_idx=0, pretrained_embeddings=None): 
        super(LSTM_QA, self).__init__()
        self.embedding_size = embedding_size
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        
        if pretrained_embeddings is None:
            self.emb = nn.Embedding(embedding_dim=self.embedding_size,num_embeddings=self.vocab_size,
                                    padding_idx=padding_idx)
        else:
            print('Loading pretrained embeddings...')
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding(embedding_dim=self.embedding_size, num_embeddings=self.vocab_size,
                                    padding_idx=padding_idx, _weight=pretrained_embeddings)
            self.emb.weight.requires_grad = False
        
        self.dropout = nn.Dropout(0.5)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, batch_first=True, dropout=0.5,bidirectional=True)
        self.conv = nn.Conv1d(in_channels=self.hidden_size*2, out_channels=10, kernel_size=3)        
        self.cosine = nn.CosineSimilarity(dim=1)
        self.linear = nn.Linear(self.hidden_size*2, 64)  
        self.linear1 = nn.Linear(64, self.n_classes)
        
        
    def forward(self, x_0, x_1):
        x_0 = self.emb(x_0)
        x_1 = self.emb(x_1)
        out_0, (ht_0, ct_0) = self.lstm(x_0)
        out_1, (ht_1, ct_1) = self.lstm(x_1)        
        x_0 = self.conv(ht_0)
        x_1 = self.conv(ht_0)
        print(x_0.shape)
        print(x_1.shape)
        
        x = self.cosine(out_0, out_1)
        x = self.linear(x)
        x = self.linear1(x)
        x = F.softmax(x, dim=0)
        return x

In [53]:
def get_optimizer(model, lr=0.01, wd=0.0):
    return torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

In [34]:
# embedding_file = "trained_models/biomedical_embeddings/Scielo_wiki_FastText300.vec"
# word_to_idx, embeddings = load_embeddings_from_file(embedding_file)

# save_dataset_to_pickle('trained_models/biomedical_embeddings/word_to_index_ir.pickle', word_to_idx)
# save_dataset_to_pickle('trained_models/biomedical_embeddings/wordvectors_ir.pickle', embeddings)

In [35]:
word_to_idx = load_dataset_from_pickle('trained_models/biomedical_embeddings/word_to_index_ir.pickle')
embeddings = load_dataset_from_pickle('trained_models/biomedical_embeddings/wordvectors_ir.pickle')

In [36]:
embedding_file = "trained_models/biomedical_embeddings/Scielo_wiki_FastText300.vec"
words = vocab.vocab2index.keys()
embedding_matrix = make_embedding_matrix(embedding_file, list(words), word_to_idx, embeddings)

In [60]:
model = LSTM_QA(vocab_size=len(vocab), hidden_size=64, x_size=trainset.max_length, n_classes=1, embedding_size=300,
               pretrained_embeddings=embedding_matrix)
optimizer = get_optimizer(model, lr = 0.001, wd = 1e-5)

Loading pretrained embeddings...


In [61]:
for x_0, x_1, y in train_dt:
    out = model(x_0.long(), x_1.long())
    print(out.shape)
    break;
    

RuntimeError: Given groups=1, weight of size [10, 128, 3], expected input[2, 32, 64] to have 128 channels, but got 32 channels instead

In [63]:
def validate_ir(model, dataloader):
    model.eval()
    loss, right, total = 0, 0, 0
    y_true, y_preds = [], []
    for x_0, x_1, y in dataloader:
        batch = y.shape[0]
        out = model(x_0.long(), x_1.long())
        loss = F.binary_cross_entropy(out, y.float())
        loss += batch*(loss.item())
        total += batch
        pred = torch.where(out > 0.4, 1, 0)
        y_true.append(y)
        y_preds.append(pred)
        right += (pred == y).float().sum().item()
    return loss/total, right/total, y_true, y_preds

In [64]:
training_results = train_ir(model, optimizer, train_dt, valid_dt, validate_ir, epochs=100)

torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([

torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([32, 1])
torch.Size([32, 128])
torch.Size([32, 64])
torch.Size([

KeyboardInterrupt: 

In [ ]:
acc, points = evaluate(model, testing, trainset.encode, evaluator_ir)
acc, points

In [ ]:
acc, points = evaluate(model, validation, trainset.encode, evaluator_ir)
acc, points

In [ ]:
model_path = os.getcwd() + '/trained_models/lstm_qa_cnn'
torch.save(model.state_dict(), model_path)